<a href="https://colab.research.google.com/github/kuakeke/113-2pgfinal/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E6%A1%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px

st.set_page_config(page_title="學習評量分析工具", layout="wide")
st.title("📊 學習評量計算機")

st.markdown("上傳學生作答的 Google Sheet（匯出成 CSV），每列為題號，每欄為學生答題紀錄（0 或 1）")

uploaded_file = st.file_uploader("請上傳 CSV 檔案", type="csv")

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    df = df.set_index("question")
    st.subheader("原始資料預覽")
    st.dataframe(df)

    df_t = df.T.copy()
    df_t["score"] = df_t.sum(axis=1)

    sorted_students = df_t.sort_values("score", ascending=False)
    n = len(sorted_students)
    group_size = round(n * 0.27)
    high_group = sorted_students.head(group_size).drop(columns="score")
    low_group = sorted_students.tail(group_size).drop(columns="score")

    high_correct_rate = high_group.T.mean(axis=1)
    low_correct_rate = low_group.T.mean(axis=1)
    difficulty = ((high_correct_rate + low_correct_rate) / 2).round(2)

    def get_difficulty_label(p):
        if p >= 0.80:
            return "極容易"
        elif 0.60 <= p < 0.80:
            return "容易"
        elif 0.40 <= p < 0.60:
            return "難易適中"
        elif 0.20 <= p < 0.40:
            return "困難"
        else:
            return "極困難"

    P = difficulty.apply(get_difficulty_label)

    high_correct_counts = high_group.T.sum(axis=1)
    low_correct_counts = low_group.T.sum(axis=1)
    discrimination = ((high_correct_counts - low_correct_counts) / (n / 2)).round(2)

    def get_discrimination_label(d):
        if d >= 0.40:
            return "極佳的試題"
        elif 0.30 <= d < 0.40:
            return "尚可的試題，可能需要稍加改進"
        elif 0.20 <= d < 0.30:
            return "不佳的試題，必須加以改進，或棄卻"
        else:
            return "極差的試題，應棄卻"

    evaluation = discrimination.apply(get_discrimination_label)

    d_df = pd.DataFrame({
        "高分組答對率": high_correct_rate,
        "高分組答對數": high_correct_counts,
        "低分組答對率": low_correct_rate,
        "低分組答對數": low_correct_counts,
        "難度": difficulty,
        "鑑別度": discrimination,
        "難易評鑑": P,
        "鑑別評鑑": evaluation
    })

    st.subheader("📋 試題分析報表")
    st.dataframe(d_df)

    # 聚類與視覺化
    d_df = d_df.reset_index().rename(columns={"index": "question"})
    kmeans = KMeans(n_clusters=4, random_state=42)
    d_df['分類群組'] = kmeans.fit_predict(d_df[["難度", "鑑別度"]])
    pca = PCA(n_components=2)
    components = pca.fit_transform(d_df[["難度", "鑑別度"]])
    d_df['主成分1'] = components[:, 0]
    d_df['主成分2'] = components[:, 1]

    st.subheader("📌 聚類視覺化：難度 vs 鑑別度")
    fig = px.scatter(
        d_df,
        x="難度",
        y="鑑別度",
        color="分類群組",
        text="question",
        title="試題分類分析圖（難度與鑑別度）",
        hover_data=["高分組答對率", "低分組答對率"]
    )
    fig.update_traces(textposition='top center')
    st.plotly_chart(fig, use_container_width=True)

    csv = d_df.to_csv(index=False).encode('utf-8-sig')
    st.download_button(
        label="📥 下載分析結果 CSV",
        data=csv,
        file_name='試題分析結果.csv',
        mime='text/csv'
    )
